In [2]:
# This module attempts to implement the DQ model of a BLDC motor following the paper here:
# https://ieeexplore.ieee.org/document/6687627

import math
import numpy as np

In [ ]:
# Motor Parameters (Table 1)
